In [2]:
# Librairies

import tensorflow as tf
import os
from PIL import Image
import numpy as np
from pandas import read_csv

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (c:\Users\xtrem\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\internal\__init__.py)

In [23]:
# Install CUDA to train on GPU (faster) and pip install tensorflow-gpu 

gpu = tf.config.list_physical_devices('GPU')
print(gpu)

if gpu:
    try:
        for gpu in gpu:
            tf.config.experimental.set_memory_growth(gpu, True)

        tf.config.experimental.set_visible_devices(gpu[0], 'GPU')
    except RuntimeError as e:
        print(e)

[]


In [2]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_image(path, img_size=256):
    img = Image.open(path).resize((img_size, img_size))
    img = np.array(img)
    img = img / 255.0
    return img


# def load_dataset(dossier_path, img_size=256):
#     images = []
#     print("Importation des images de ", dossier_path,"...")
#     for img_name in os.listdir(dossier_path):
#         if not img_name.endswith('.jpg'):
#             print(img_name," n'est pas une photo")

#         img = load_image(os.path.join(dossier_path, img_name), img_size)
#         images.append(img)
            
#     print(len(images), " images importées")
#     return np.array(images)


def load_dataset(directory, img_size=256):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"): 
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=(img_size, img_size))
            img = img_to_array(img) / 127.5 - 1.0 
            images.append(img)
    print(f"{len(images)} images importées depuis {directory}")
    return np.array(images)

def get_label(img_name, labels=None):
    if labels is None:
        labels = read_csv('dataset/metadata.csv')
    a = labels.loc[labels['image_path'] == img_name]

    if not a.empty:
        return a['domain'].values[0]
    else:
        return None

In [3]:
trainA = load_dataset("dataset/trainA")
# trainB = load_dataset("dataset/trainB")
testA = load_dataset("dataset/testA")
# testB = load_dataset("dataset/testB")


print(trainA.shape, trainA[0].shape)
get_label("trainA/n02381460_2.jpg")

1067 images importées depuis dataset/trainA
35 images importées depuis dataset/testA
(1067, 256, 256, 3) (256, 256, 3)


'A (Horse)'

In [4]:
from build_gan import *

In [6]:
trainA = load_dataset("dataset/trainA")
img_shape = trainA.shape[1:]
print(img_shape)


latent_dim = 100  # Dimension de bruit
generator = build_generator(latent_dim)
discriminator = build_discriminator(img_shape)

# generator.summary()
# discriminator.summary()

1067 images importées depuis dataset/trainA
(256, 256, 3)


In [7]:
gan = build_gan(generator, discriminator)

In [8]:
learning_rate = 0.0002
beta_1 = 0.5

discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate, beta_1), metrics=['accuracy'])
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate, beta_1))

In [10]:
train_gan(generator, discriminator, gan, trainA, epochs=5000, batch_size=32, modulo_epoch_display=10) # train on GPU with CUDA...

1/1 [==============================] - 0s 282ms/step
0 [D loss: 0.7047606706619263, acc.: 18.75] [G loss: 0.6839777231216431]
1/1 [==============================] - 0s 229ms/step
10 [D loss: 0.7468516230583191, acc.: 21.875] [G loss: 0.608108639717102]
1/1 [==============================] - 0s 241ms/step
20 [D loss: 0.897643119096756, acc.: 25.0] [G loss: 0.4066259562969208]
1/1 [==============================] - 0s 242ms/step


In [12]:

# 30 epochs = 3min so i put that to not re-train each time bc 10000 epochs = 15h on CPU
generator.save("models/generator.h5")
discriminator.save("models/discriminator.h5")
gan.save("models/gan.h5")

In [16]:
import matplotlib.pyplot as plt

def generate_and_save_images(generator, nb_images=100, save_dir='generated_images'):
    os.makedirs(save_dir, exist_ok=True)
    noise = np.random.normal(0, 1, (nb_images, 100))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5  # Re-scaler les images

    for i in range(nb_images):
        plt.imsave(f"{save_dir}/generated_image_{i}.png", gen_imgs[i])

generate_and_save_images(generator, 20, 'gen_images')

4/4 [==============================] - 1s 310ms/step
